In [ ]:
import boto3
import sagemaker
from sagemaker import Model, serializers, deserializers

# role = sagemaker.get_execution_role()  # execution role for the endpoint
role = 'arn:aws:iam::633205212955:role/service-role/AmazonSageMaker-ExecutionRole-20220923T160810'

sess = sagemaker.session.Session()
region = sess._region_name
account_id = sess.account_id()

full_image_uri = '633205212955.dkr.ecr.us-east-1.amazonaws.com/sglang-inf-docker-lmsys:latest'
print(full_image_uri)

/opt/conda/envs/pydev/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[01/13/25 01:18:21] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=327288;file:///opt/conda/envs/pydev/lib/python3.11/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=937009;file:///opt/conda/envs/pydev/lib/python3.11/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ubuntu/.config/sagemaker/config.yaml


                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=755240;file:///opt/conda/envs/pydev/lib/python3.11/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=967422;file:///opt/conda/envs/pydev/lib/python3.11/site-packages/botocore/credentials.py#1278\1278]8;;\

633205212955.dkr.ecr.us-east-1.amazonaws.com/sglang-inf-docker-lmsys-6:latest


In [ ]:
# model = Model(image_uri=full_image_uri, model_data=code_artifact, role=role)
model = Model(role=role, image_uri=full_image_uri, 
              env={
                # "MODEL_ID_OR_S3_PATH": "NousResearch/Meta-Llama-3-8B-Instruct",
                "MODEL_ID_OR_S3_PATH": "s3://llm-artifacts-us-east-1/Llama-3-8B-Instruct/",
                "CONTEXT_LEN": "4096",
                "MEM_FRAC": "0.8",
                "SES_TTL_MIN": str(60*24) ## 15minute max
        })

endpoint_name = sagemaker.utils.name_from_base('stateful-sanic-v6-g6e-2xl')
instance_type = 'ml.g5.2xlarge'

model.deploy(initial_instance_count=4,
             instance_type=instance_type,
             endpoint_name=endpoint_name
            )

[01/13/25 01:05:29] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=37293;file:///opt/conda/envs/pydev/lib/python3.11/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=147376;file:///opt/conda/envs/pydev/lib/python3.11/site-packages/botocore/credentials.py#1278\1278]8;;\

                    INFO     Creating model with name:                                              ]8;id=482402;file:///opt/conda/envs/pydev/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=865306;file:///opt/conda/envs/pydev/lib/python3.11/site-packages/sagemaker/session.py#4094\4094]8;;\
                             sglang-inf-docker-lmsys-6-2025-01-13-01-05-29-677                                     

[01/13/25 01:05:30] INFO     Creating endpoint-config with name                                     ]8;id=660874;file:///opt/conda/envs/pydev/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=920786;file:///opt/conda/envs/pydev/lib/python3.11/site-packages/sagemaker/session.py#5889\5889]8;;\
                             stateful-sanic-v6-g6e-2xl-2025-01-13-01-05-29-436                                     

                    INFO     Creating endpoint with name                                            ]8;id=161762;file:///opt/conda/envs/pydev/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=6212;file:///opt/conda/envs/pydev/lib/python3.11/site-packages/sagemaker/session.py#4711\4711]8;;\
                             stateful-sanic-v6-g6e-2xl-2025-01-13-01-05-29-436                                     

--

In [3]:
# our requests and responses will be in json format so we specify the serializer and the deserializer

endpoint_name = 'stateful-sanic-v6-g6e-2xl-2025-01-13-01-05-29-436'
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=serializers.JSONSerializer(),
    deserializer=deserializers.JSONDeserializer(),
)

In [4]:
predictor.predict(
    {"inputs": "Deep Learning is", "parameters": {"max_new_tokens":128, "temperature":0.2}}
)

{'generation': {'text': ' a subfield of Machine Learning that involves the use of Artificial Neural Networks (ANNs) to model complex patterns in data. Deep Learning models are designed to learn hierarchical representations of data, which enables them to capture subtle patterns and relationships that may not be apparent through other machine learning techniques.\n\nDeep Learning has been successful in a wide range of applications, including:\n\n1. Computer Vision: Deep Learning models have achieved state-of-the-art results in image classification, object detection, segmentation, and generation tasks.\n2. Natural Language Processing: Deep Learning models have been used to build language translation systems, chatbots, and text summarization systems.\n3.',
  'meta_info': {'prompt_tokens': 4,
   'completion_tokens': 128,
   'completion_tokens_wo_jump_forward': 128,
   'cached_tokens': 0,
   'finish_reason': {'type': 'length', 'length': 128},
   'id': '8a72373d64424446b07de65de19fa652'}}}

In [ ]:
predictor.predict(
    {"inputs": "Deep Learning is", "parameters": {"max_new_tokens":8, "temperature":0.9}}
)

In [ ]:
import boto3
import json

sm_bt3_client = boto3.client("runtime.sagemaker")
response = sm_bt3_client.invoke_endpoint(
                EndpointName=endpoint_name,
                Body=json.dumps({"inputs": "Deep Learning is", "parameters": {"max_new_tokens":128, "temperature":0.2}}),
                ContentType="application/json"
        )

In [ ]:
response